# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [6]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [7]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [8]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [9]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [10]:
# make a connection to a Cassandra instance
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# establish connection
session = cluster.connect()

#### Create Keyspace

In [11]:
# Create a Keyspace 
try:
    session.execute(""" create keyspace if not exists udacity
                         with replication=
                         {'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [12]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4



* We created a table with the columns:`sessionIs, itemInSession,artist,song_title,song_length`
* Since we are querying based on sessionid and iteminsession, we will have these two columns as our primary key and can uniquely identify our rows.
* Here `sessionid` will be our partition key and `iteminsession` will be our clustering key. Having alone `sessionid` as partition key in primary key will not work as we may have same rows with sessionid. So, adding `iteminsession` as clusterkey will make unique for each row. 


In [13]:
# Create the table for query1
session.execute("""drop table if exists artist_song;""")
session.execute("""create table if not exists artist_song(sessionId int, \
                                       itemInSession int, \
                                       artist varchar, \
                                       song_title varchar, \
                                       song_length float, \
                                       primary key((sessionId),itemInSession));""")

In [14]:
# reading the CSV file and Inserting the rows in the above created table(ETL pipline)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into artist_song(sessionId,itemInSession,artist,song_title,song_length)"
        query = query + "values(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0],line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
out=session.execute("select artist,song_title,song_length from artist_song where sessionId=338 and itemInSession=4;")
for i in out:
    print('{}\t{}\t{}\t'.format(i.artist,i.song_title,i.song_length))

Faithless	Music Matters (Mark Knight Dub)	495.30731201171875	


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 18


* We created the table with the columns: `userid, sessionid, iteminsession,artist, song, firstname, lastname`.
* Here are we are quering on userid, sessionid and we are asked to sort by iteminsession we will have all three in primary key.
* Here we will have `userid` and `sessionid` as our partition key.Having alone `userid` and `sessionid` as primary key will not work because many rows will have same userid and sessionid .Including the iteminsession as clustered key will make each row unique and sort based on clustered key

In [17]:
# Create the table for query2
session.execute("drop table if exists artist_user;")
session.execute("create table if not exists artist_user(userId int, \
                                                        sessionId int, \
                                                        itemInSession int, \
                                                        artist varchar, \
                                                        song varchar, \
                                                        firstname varchar, \
                                                        lastname varchar, \
                                                        primary key((userId,sessionId),itemInSession));")
                    

In [18]:
# reading the CSV file and Inserting the rows in the above created table(ETL pipline)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into artist_user(userId,sessionId,itemInSession,artist,song,firstname,lastname)"
        query = query + "values(%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [19]:
out=session.execute("select artist,song,firstname,lastname from artist_user where userId=10 and sessionId=182;")
for i in out:
    print('{}\t{}\t{}\t{}'.format(i.artist,i.song,i.firstname,i.lastname))

Down To The Bone	Keep On Keepin' On	Sylvie	Cruz
Three Drives	Greece 2000	Sylvie	Cruz
Sebastien Tellier	Kilometer	Sylvie	Cruz
Lonnie Gordon	Catch You Baby (Steve Pitron & Max Sanna Radio Edit)	Sylvie	Cruz


    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


* Here we created tabel with the columns: 
`song, userid, firstname, lastname`.
* Since we are querying for `song` we can have it in primary key but it alone will not work as many users can listen to it. So adding the `userid` will make each row unique.
* We have `song` as the the partition key and `userid` as clustered key 

In [20]:
# Create table for query3
session.execute("drop table if exists username;")
session.execute("create table username(song varchar, \
                                       userid int, \
                                       firstname varchar, \
                                       lastname varchar, \
                                       primary key((song),userid));")
                    

In [22]:
# reading the CSV file and Inserting the rows in the above created table(ETL pipline)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into username(song,userid,firstname,lastname)"
        query = query + "values(%s,%s,%s,%s)"
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
out=session.execute("select firstname,lastname from username where song='All Hands Against His Own';")
for i in out:
    print('{}\t{}'.format(i.firstname,i.lastname))

Jacqueline	Lynch
Tegan	Levine
Sara	Johnson


### Drop the tables before closing out the sessions

In [28]:
session.execute("drop table if exists artist_song;")
session.execute("drop table if exists artist_user")
session.execute("drop table if exists ")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()